In [ ]:
import numpy as np
from pandas import read_csv
from javelin.structure import Structure
from custom.inout import read_structure

# path to the folder containing .csv input files generated by Ising.cpp
input_path = "structure"
#number of imported samples 
n_samples = 10
samples = []



"""Reading structures with relaxed Bi-O distances """

for i in range(n_samples):
    samples.append(read_structure(input_path+"_"+str(i)+".txt"))
samples[0].atoms



In [ ]:

""" Calculating average Bi3+-O and Bi5+-O bond-length after MC relaxation """
def average_distance(structure): 
    x = structure.atoms.xs(1, level='site').x.to_numpy()
    y = structure.atoms.xs(2, level='site').y.to_numpy()
    z = structure.atoms.xs(3, level='site').z.to_numpy()
    Bi =structure.atoms.xs(0, level='site').Z.to_numpy()

    x3 = x[Bi == 80]
    y3 = y[Bi == 80]
    z3 = z[Bi == 80]

    x5 = x[Bi == 78]
    y5 = y[Bi == 78]
    z5 = z[Bi == 78]

    av3 = (x3.sum()/x3.size + y3.sum()/y3.size + z3.sum()/z3.size)/3
    av5 = (x5.sum()/x5.size + y5.sum()/y5.size + z5.sum()/z5.size)/3

    return av3, av5 

D1 = 0
D2 = 0
for i in range(n_samples):
    D1 = D1 + average_distance(samples[i])[0]
    D2 = D2 + average_distance(samples[i])[1]

D1 = D1/n_samples
D2 = D2/n_samples


print("Bi3+ - O distance: ",  D1) 
print("Bi5+ - O distance: ",  D2) 

In [ ]:
""" 
Writing input files for scatty -> To calculate diffuse scattering run scatty in
directory containing input files (see ReadMe.txt in Dependencies/Scatty subfolder)
"""

from custom.inout import Scatty as SC 
sc = SC() 
sc.basis = [(0.0,0.0,0.0),(0.5,0.0,0.0),(0.0,0.5,0.0),(0.0,0.0,0.5),(0.5,0.5,0.5)]
sc.centre = (0,0,0.5)
sc.x_axis = (6,0,0,600)
sc.y_axis = (0,6,0,600)
sc.z_axis = (0,0,0,1)
sc.remove_bragg = False
sc.cell_centering = "P"
sc.expansion_error = 0.001
sc.expansion_order = 10
sc.window = 2
sc.cutoff = 2
sc.ppm_output = True
sc.ppm_colourmap = "viridis"
sc.radiation = "X"
sc.symmetry = "m3m"
sc.path = ""
sc.title = "cdw"
sc.name = "L=0.5"
sc.replacement_dict = {"Pt":"Bi5+", "Hg":"Bi3+"}
sc.write_input(samples)